# 데이터 불러오기

In [17]:
import pandas as pd
import numpy as np

df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/검색어 수.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   인기 검색어  1000 non-null   object 
 1   클릭수     1000 non-null   int64  
 2   노출      1000 non-null   int64  
 3   CTR     1000 non-null   object 
 4   게재 순위   1000 non-null   float64
dtypes: float64(1), int64(2), object(2)
memory usage: 39.2+ KB


## 데이터 분석

### 키워드 점유 현황은?
-> 도메인과 관련 있는 키워드 점유 현황을 바탕삼아 콘텐츠 발행 성과 점검 가능

- 게재 순위 소수점 제거

In [63]:
### 게재 순위 소수점 제거

df['게재 순위'] = df['게재 순위'].astype('int')
# df.info()
df.head()

,인기 검색어,클릭수,노출,CTR,게재 순위,순위 분류
0,2024년 음주운전 초범 벌금,427,1968,21.70,2,top3
1,만 16세 이상 합의,380,1984,19.15,2,top3
2,온리팬스 처벌,200,3633,5.51,3,top3
3,트위터 간단 경찰,185,1209,15.30,1,top3
4,주거침입죄 초범,166,647,25.66,1,top3


- 키워드 순위별 분류

In [64]:
# 순위 분류할 열 생성
df['순위 분류'] = '기타'
df.head()
# 생성한 열에 조건에 따라 게재 순위 넣기

import numpy as np

# 3위 안에 들어간 키워드
df['순위 분류'] = np.where(df['게재 순위'] <= 3, 'top3', df['순위 분류'])
# 3위 안은 아니지만, 10위 안에 들어간 키워드
df['순위 분류'] = np.where((df['게재 순위'] > 3) & (df['게재 순위'] <= 10), 'top10', df['순위 분류'] )
# 10위 안은 아니지만, 20위 안에 들어간 키워드
df['순위 분류'] = np.where((df['게재 순위'] > 10) & (df['게재 순위'] <= 20), 'top20', df['순위 분류'] )
# df.head()
df['순위 분류'].value_counts()

,count
순위 분류,
top10,373
기타,280
top20,222
top3,125


### 키워드 순위별 평균 노출수·평균 클릭률은?
-> 키워드 품질을 평가할 수 있는 '기준'을 만들기 위해서

In [65]:
### 키워드 순위별 평균 노출수·평균 클릭률은?

# 키워드 순위별 인기 검색어 수
df.groupby('순위 분류')['인기 검색어'].count()

# 키워드 순위별 평균 노출·클릭률은?
  # CTR 데이터타입이 %가 포함되어있어서 object.
    # 변환 작업을 해야함.
df.groupby('순위 분류')[['노출', 'CTR']].agg({'노출': 'mean', 'CTR': 'mean'})

,노출,CTR
순위 분류,,
top10,93.922252,4.575201
top20,49.072072,5.720631
top3,190.144000,11.325280
기타,16.682143,7.614893


In [66]:
# CTR 데이터타입 변경
# df['CTR'] = df['CTR'].str.replace('%','').astype('float')

# 키워드 순위별 평균 노출수·클릭률은?
키워드평균노출클릭률 = df.groupby('순위 분류')[['노출', 'CTR']].agg({'노출' : 'mean', 'CTR' : 'mean'})

In [67]:
# 데이터타입 변경
  # 변경한 후에 반드시 다시 해당 컬럼에 적용값을 적용해 줘야 한다.

키워드평균노출클릭률['노출'] = 키워드평균노출클릭률['노출'].astype('int')
키워드평균노출클릭률.info()

키워드평균노출클릭률['CTR'] = 키워드평균노출클릭률['CTR'].astype('int')
키워드평균노출클릭률.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4 entries, top10 to 기타
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   노출      4 non-null      int64  
 1   CTR     4 non-null      float64
dtypes: float64(1), int64(1)
memory usage: 96.0+ bytes
<class 'pandas.core.frame.DataFrame'>
Index: 4 entries, top10 to 기타
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype
---  ------  --------------  -----
 0   노출      4 non-null      int64
 1   CTR     4 non-null      int64
dtypes: int64(2)
memory usage: 96.0+ bytes


In [68]:
키워드평균노출클릭률['CTR'] = 키워드평균노출클릭률['CTR'].apply(lambda x : f'{x: .1f}%')
키워드평균노출클릭률.head()

,노출,CTR
순위 분류,,
top10,93,4.0%
top20,49,5.0%
top3,190,11.0%
기타,16,7.0%


### 웹사이트 트래픽을 불러일으키는 top10 웹페이지 및 키워드는?
-> 1.성과로 이어진 키워드를 확인하기 위함.
2. 성과로 이어진 키워드와 비슷한 수치를 가졌지만, 성과로 이어지지 못한 키워드 확인하기 위함.

In [37]:
### 웹사이트 트래픽을 불러일으키는 top10 웹페이지 및 키워드는?

# 데이터 로드
df1 = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/검색어 수.csv')
df2 = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/페이지.csv')

# 데이터 조인
mergr_df = pd.merge(df1, df2, on='클릭수', how='left')
mergr_df.head(10)

,인기 검색어,클릭수,노출_x,CTR_x,게재 순위_x,인기 페이지,노출_y,CTR_y,게재 순위_y
0,2024년 음주운전 초범 벌금,427,1968,21.7%,2.71,NaN,NaN,NaN,NaN
1,만 16세 이상 합의,380,1984,19.15%,2.63,NaN,NaN,NaN,NaN
2,온리팬스 처벌,200,3633,5.51%,3.52,NaN,NaN,NaN,NaN
3,트위터 간단 경찰,185,1209,15.3%,1.51,NaN,NaN,NaN,NaN
4,주거침입죄 초범,166,647,25.66%,1.03,NaN,NaN,NaN,NaN
5,온리팬스 결제 처벌,161,2693,5.98%,3.35,NaN,NaN,NaN,NaN
6,성매수자 처벌,151,1901,7.94%,4.86,NaN,NaN,NaN,NaN
7,온리팬스 불법,146,1566,9.32%,3.40,NaN,NaN,NaN,NaN
8,성범죄 합의해도 처벌,98,390,25.13%,1.75,NaN,NaN,NaN,NaN
9,음주운전 벌금 언제 나오나요,94,985,9.54%,6.60,NaN,NaN,NaN,NaN


- 웹사이트 트래픽을 불러일으키는 top10 키워드

In [106]:
## 웹사이트에 트래픽을 유도하는 키워드는?

# 유니크한 값 및 개수 세기
# mergr_df['인기 검색어'].value_counts()

# 중복되는 값 직접 제거하기
# mergr_df.drop_duplicates(subset=['인기 검색어'], inplace=True)

# 제거된 결과 확인하기
mergr_df['인기 검색어'].value_counts()

,count
인기 검색어,
2024년 음주운전 초범 벌금,1
전치2주합의금,1
카촬양형,1
성추행 사례,1
시청만 해도 처벌,1
...,...
초범 벌금,1
성희롱 처벌 사례,1
성추행 증거,1


In [111]:
mergr_df[['인기 검색어', '클릭수', '노출_x']].head(10)

,인기 검색어,클릭수,노출_x
0,2024년 음주운전 초범 벌금,427,1968
1,만 16세 이상 합의,380,1984
2,온리팬스 처벌,200,3633
3,트위터 간단 경찰,185,1209
4,주거침입죄 초범,166,647
5,온리팬스 결제 처벌,161,2693
6,성매수자 처벌,151,1901
7,온리팬스 불법,146,1566
8,성범죄 합의해도 처벌,98,390
9,음주운전 벌금 언제 나오나요,94,985


In [114]:
# mergr_df['인기 페이지'].isna().sum()
mergr_df['인기 페이지'].value_counts()
mergr_df.drop_duplicates(subset=['인기 페이지'], inplace=True)

,count
인기 페이지,
https://criminal-lawfirm-dongju.com/Sex-Offenses-Column/club-grinding-assault-punishment,1
https://criminal-lawfirm-dongju.com/Sex-Offenses-Column/military-sexual-crime-punishment,1
https://criminal-lawfirm-dongju.com/Drunk-accident-column/dui-revocation-of-license,1
https://criminal-lawfirm-dongju.com/dongju-Introduction/오서진,1
https://criminal-lawfirm-dongju.com/Drunk-accident-column/drunk-driving-revocation-of-license,1
https://criminal-lawfirm-dongju.com/Drunk-accident-column/dui-license-recause,1
https://criminal-lawfirm-dongju.com/Drunk-accident-column/drunk-driving-license,1
https://criminal-lawfirm-dongju.com/Sex-Offenses-Column/performance-obscene-crime-exposure-punishment,1
https://criminal-lawfirm-dongju.com/Drinking-Accidents-Successful-Cases/2024-success-dui-fine-avoidance,1


- 웹사이트 트래픽을 불러일으키는 top10 키워드 및 웹페이지

In [120]:
# mergr_df[['인기 검색어', '클릭수', '노출_x', '인기 페이지']].head(10)

# '클릭수'를 기준으로 내림차순
mergr_df[['인기 검색어', '클릭수', '노출_x', '인기 페이지']].sort_values(by=['클릭수'], ascending=False).head(10)

# '노출수'를 기준으로 내림차순
# mergr_df[['인기 검색어', '클릭수', '노출_x', '인기 페이지']].sort_values(by=['노출_x'], ascending=False).head(10)

,인기 검색어,클릭수,노출_x,인기 페이지
0,2024년 음주운전 초범 벌금,427,1968,NaN
12,성추행 신고 당하면,77,384,https://criminal-lawfirm-dongju.com/Sex-Offens...
13,트위터 간단 단속,75,507,https://criminal-lawfirm-dongju.com/칼럼-성범죄/por...
19,만 17세 합의 관계,48,489,https://criminal-lawfirm-dongju.com/Sex-Offens...
23,미성년자 합의 관계,34,520,https://criminal-lawfirm-dongju.com/Drunk-acci...
26,음주운전 초범 벌금,33,899,https://criminal-lawfirm-dongju.com/Sex-Offens...
30,미성년자 성관계,31,411,https://criminal-lawfirm-dongju.com/Drunk-acci...
34,트위터 조건 단속,27,278,https://criminal-lawfirm-dongju.com/Sex-Offens...
39,온리팬스,24,6107,https://criminal-lawfirm-dongju.com/Sex-Offens...
40,만16세 합의 디시,22,567,https://criminal-lawfirm-dongju.com/Drunk-acci...


### 키워드별 영향을 주는 웹페이지는?

In [121]:
import pandas as pd
import numpy as np

merge_검색어 = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/검색어 수.csv')
merge_페이지 = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/페이지.csv')

merge_키워드별웹페이지 = pd.merge(merge_검색어, merge_페이지, on='클릭수', how='left')

merge_키워드별웹페이지.head()

,인기 검색어,클릭수,노출_x,CTR_x,게재 순위_x,인기 페이지,노출_y,CTR_y,게재 순위_y
0,2024년 음주운전 초범 벌금,427,1968,21.7%,2.71,NaN,NaN,NaN,NaN
1,만 16세 이상 합의,380,1984,19.15%,2.63,NaN,NaN,NaN,NaN
2,온리팬스 처벌,200,3633,5.51%,3.52,NaN,NaN,NaN,NaN
3,트위터 간단 경찰,185,1209,15.3%,1.51,NaN,NaN,NaN,NaN
4,주거침입죄 초범,166,647,25.66%,1.03,NaN,NaN,NaN,NaN


In [133]:

# apply(list)를 이용해서 한 개 키워드가 여러 웹페이지에 영향을 주는지 알 수 있음
  # 만약 apply(list)를 사용하지 않으면, 한 개 키워드가 2개 이상 웹페이지에 영향을 줄 수 있는지 여부를 알 수 없음
merge_키워드별웹페이지.groupby(['인기 검색어', '클릭수', '노출_x'])['인기 페이지'].apply(list)

# 그룹바이한 결과를 다시 클릭수로 내림차 순 적용
# merge_키워드별웹페이지.groupby(['인기 검색어', '클릭수', '노출_x'])['인기 페이지'].apply(list).reset_index().sort_values(by=['클릭수'], ascending=False)

,,,인기 페이지
인기 검색어,클릭수,노출_x,
0.03 음주,0,9,[https://criminal-lawfirm-dongju.com/Drunk-acc...
0.08 음주,0,7,[https://criminal-lawfirm-dongju.com/Drunk-acc...
16세 미만,0,6,[https://criminal-lawfirm-dongju.com/Drunk-acc...
16세미만,0,5,[https://criminal-lawfirm-dongju.com/Drunk-acc...
17살 성관계,2,35,[https://criminal-lawfirm-dongju.com/dongju-In...
...,...,...,...
황윤서 변호사,0,23,[https://criminal-lawfirm-dongju.com/Drunk-acc...
휴대폰 포렌식,1,4,[https://criminal-lawfirm-dongju.com/Drunk-acc...
휴대폰 포렌식 복구 범위,7,171,[https://criminal-lawfirm-dongju.com/Drinking-...


## FAQ

### 1. 구글서치콘솔 데이터에서 NaN이 있는 이유는?

**출처**
https://support.google.com/webmasters/answer/96568?hl=ko



**데이터 제공 기준**

1. **대표 데이터 중심**
   - 웹사이트에 중요한 영향을 미치는 데이터만 제공
   - 모든 데이터를 포함하지 않음

2. **개인정보 보호**
   - 개인정보나 민감한 정보와 관련된 데이터 제외

3. **데이터 제한**
   - Google Search Console UI에서 최대 1,000개 행만 표시

**데이터 구조**

- 위 기준에 따라 데이터가 여러 테이블로 분리됨
- 각 테이블은 서로 다른 기준과 범위의 데이터 포함

**NaN 발생 원인**

1. **테이블 조인**
   - 서로 다른 기준으로 필터링된 테이블 조인 시 NaN 발생 가능성 높음

2. **데이터 제외 기준 차이**
   - URL, 검색어, 날짜 등에 대한 데이터 제외 기준이 테이블마다 다름
   - 조인 시 불일치로 인한 NaN 발생